In [1]:
from train_utils import DrivingData
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.io import output_notebook
import ipywidgets
import numpy as np
import bokeh.plotting as bkp
from bokeh.io import output_notebook, push_notebook
# 读取数据
path = ["/cailiu2/Diffusion-Planner/data/processed/us-nv-las-vegas-strip_7fc811bcf45f5e79.npz"]
output_notebook()

train_set = DrivingData(path, 10, 10)
ego, neighbors, map_lanes, _, _, _, _, _, _ = train_set[0]

# 确保数据维度正确
print("ego shape:", ego.shape)  # 应为 (num_frames, 2)
print("neighbors shape:", neighbors.shape)  # 应为 (num_neighbors, num_frames, 2)

num_frames = len(ego)
num_neighbors = len(neighbors)

# 提取ego和neighbors的信息
ego_x, ego_y = ego[:, 0], ego[:, 1]
neighbors_x, neighbors_y = neighbors[:, :, 0], neighbors[:, :, 1]

# 提取map lanes信息
num_lanes = len(map_lanes)
num_points_of_lane = len(map_lanes[0])

lane_xs=[map_lanes[i, :, 0].tolist() for i in range(num_lanes)]
lane_ys=[map_lanes[i, :, 1].tolist() for i in range(num_lanes)]

# 初始视图范围
initial_x, initial_y = ego_x[0], ego_y[0]
p = bkp.figure(
    title="Ego Vehicle & Neighbors",
    x_range=(initial_x - 50, initial_x + 50),
    y_range=(initial_y - 50, initial_y + 50),
    width=600, height=600
)

# 数据源
source = ColumnDataSource(data={
    'ego_x': [],
    'ego_y': [],
})

agents_source = ColumnDataSource(data={
    'neighbors_x': neighbors_x[:, 0].tolist(),
    'neighbors_y': neighbors_y[:, 0].tolist()
})

map_lane_source = ColumnDataSource(data={'lane_xs': lane_xs, 'lane_ys': lane_ys})
# 绘图
p.circle('ego_x', 'ego_y', source=source, size=12, color="red", legend_label="Ego Vehicle")
p.scatter('neighbors_x', 'neighbors_y', source=agents_source, size=6, color="blue", alpha=0.6, legend_label="Neighbors")

p.multi_line('lane_xs', 'lane_ys', source=map_lane_source, line_width=2, color="blue", alpha=0.6)
# 创建滑块类，初始化并绑定回调
slider_class = ipywidgets.IntSlider(
    value=0, min=0, max=num_frames-1, step=1, description="Frame"
)

# 滑块回调
def slider_callback(frame):
    print("frame: ", frame)
    new_ego_x = ego_x[frame]
    new_ego_y = ego_y[frame]
    print("new ego_x: ", new_ego_x)
    print("new ego_y: ", new_ego_y)

    # 更新数据
    source.data.update({'ego_x': [new_ego_x], 'ego_y': [new_ego_y]})
    agents_source.data.update( {'neighbors_x': neighbors_x[:, frame].tolist(), 'neighbors_y': neighbors_y[:, frame].tolist()})
    bkp.show(p, notebook_handle=True)

push_notebook()

ipywidgets.interactive(slider_callback, frame=slider_class)




Loading BokehJS ...

ego shape: (21, 7)
neighbors shape: (20, 21, 11)


/root/miniconda3/envs/diffusion_planner/lib/python3.9/site-packages/bokeh/io/notebook.py:311: UserWarning: Cannot find a last shown plot to update. Call output_notebook() and show(..., notebook_handle=True) before push_notebook()
  warn("Cannot find a last shown plot to update. Call output_notebook() and show(..., notebook_handle=True) before push_notebook()")


interactive(children=(IntSlider(value=0, description='Frame', max=20), Output()), _dom_classes=('widget-intera…

In [2]:
print(map_lanes.shape)
num_lanes = len(map_lanes)
num_points_of_lane = len(map_lanes[0])
print(num_lanes)
print(num_points_of_lane)

(40, 50, 7)
40
50
